In [2]:
import findspark
#useful to install this tool to simplify spark import
findspark.init()
from pyspark import  SparkContext
sc = SparkContext( 'local[*]', 'pyspark')
import numpy

In [4]:
#filtering items to obtain only which have a ratings by test_users
test_rdd = sc.textFile("test_no_header.csv")
test_users=test_rdd.collect()[0:99]

def test_item_filter(x):
    for i in x[1]:
        if i[0] in test_users:
            return True
    return False

test_item_votes_rdd=(sc
                     .textFile("train_no_header.csv")
                     .map(lambda x: x.split(','))
                     .map(lambda x:(x[1],[x[0],int(x[2])]))
                     .groupByKey()
                     .filter(test_item_filter)
                     .map(lambda x: (x[0],list(x[1]))))
                    
              
test_item_votes_rdd.take(10)  
test_item_votes_rdd.count()

909

In [5]:
rdd = sc.textFile("train_no_header.csv")
rdd = rdd.map( lambda x: x.split(',') )
rdd = rdd.map(lambda x:(x[1],[x[0],int(x[2])]))

rdd = rdd.groupByKey()
rdd = rdd.map(lambda x:(x[0],list(x[1])))

rdd.takeOrdered(10)

[(u'1', [[u'2738', 1], [u'4716', 4], [u'13298', 8], [u'15122', 4]]),
 (u'10',
  [[u'3136', 5],
   [u'5351', 6],
   [u'8218', 4],
   [u'8295', 4],
   [u'9859', 2],
   [u'10674', 5],
   [u'13868', 6]]),
 (u'100',
  [[u'122', 9],
   [u'620', 6],
   [u'738', 10],
   [u'1236', 9],
   [u'1437', 10],
   [u'1853', 8],
   [u'2145', 10],
   [u'2682', 7],
   [u'3088', 7],
   [u'3288', 5],
   [u'5490', 8],
   [u'5704', 10],
   [u'6235', 8],
   [u'6344', 9],
   [u'6901', 10],
   [u'7393', 10],
   [u'7943', 7],
   [u'8246', 9],
   [u'9052', 10],
   [u'9291', 10],
   [u'10040', 9],
   [u'10847', 7],
   [u'12072', 9],
   [u'12593', 8],
   [u'13065', 8],
   [u'13098', 8],
   [u'13484', 8],
   [u'13591', 5]]),
 (u'1000', [[u'7924', 3]]),
 (u'10003', [[u'2128', 8]]),
 (u'10004', [[u'4716', 8], [u'11752', 8], [u'15028', 7]]),
 (u'10005', [[u'7393', 6], [u'9346', 2]]),
 (u'10006', [[u'3664', 7], [u'7913', 4]]),
 (u'10007', [[u'6738', 10], [u'7665', 8], [u'9370', 1]]),
 (u'10008', [[u'1204', 5]])]

In [6]:
from Similarities import sim_pearson


sim_rdd=test_item_votes_rdd.cartesian(rdd)
sim_rdd=sim_rdd.map(lambda x: (x[0][0],[x[1][0],sim_pearson(x[1][1],x[0][1])])).filter(lambda x:x[1][1]>0 and x[1][0]!=x[0])
sim_rdd.take(10)

[(u'36354', [u'876', 1.0]),
 (u'36354', [u'29642', 1.0]),
 (u'36354', [u'25442', 1.0]),
 (u'36354', [u'3700', 1.0]),
 (u'36354', [u'6446', 1.0]),
 (u'36354', [u'6314', 1.0]),
 (u'36354', [u'5889', 1.0]),
 (u'36354', [u'21116', 1.0]),
 (u'36354', [u'16484', 1.0]),
 (u'36354', [u'7597', 1.0])]

In [5]:
#sim2_rdd=sim_rdd.map(lambda x:(x[1][0],[x[0],x[1][1]]))
#total_sim_rdd=sim_rdd.union(sim2_rdd)
#total_sim_rdd.take(10)

KeyboardInterrupt: 

In [7]:
test_rdd = sc.textFile("test_no_header.csv")
test_users=test_rdd.collect()[0:99]

rev_urm_rdd=(sc
         .textFile("train_no_header.csv")
         .map(lambda x: x.split(','))
         .filter(lambda x:x[0] in test_users)
         .map(lambda x:(x[1],[x[0],int(x[2])])))


#urm_rdd.takeOrdered(10,key=lambda x: int(x[0]))

#test_urm_rdd=test_rdd.join(urm_rdd)
#test_urm_rdd.take(10)
rev_urm_rdd.take(10)

[(u'30', [u'152', 5]),
 (u'100', [u'122', 9]),
 (u'228', [u'151', 9]),
 (u'269', [u'111', 10]),
 (u'269', [u'205', 10]),
 (u'371', [u'152', 7]),
 (u'418', [u'173', 9]),
 (u'562', [u'93', 7]),
 (u'573', [u'152', 7]),
 (u'603', [u'321', 9])]

In [8]:
print rev_urm_rdd.count()


959


In [17]:

vote_rdd=rev_urm_rdd.join(sim_rdd)


          #.map(compute_rec)
          #.groupByKey()
          #.map(lambda x: (x[0],list(y for y in x[1]))))
         


In [18]:
vote_rdd.take(10)

[(u'15452', ([u'152', 5], [u'30925', 0.802955068546975])),
 (u'15452', ([u'152', 5], [u'21077', 1.0])),
 (u'15452', ([u'152', 5], [u'21613', 1.0])),
 (u'15452', ([u'152', 5], [u'3515', 0.7778944485132251])),
 (u'15452', ([u'152', 5], [u'8297', 1.0])),
 (u'15452', ([u'152', 5], [u'3742', 1.0])),
 (u'15452', ([u'152', 5], [u'10576', 1.0])),
 (u'15452', ([u'152', 5], [u'23756', 0.8660254037844448])),
 (u'15452', ([u'152', 5], [u'16916', 1.0])),
 (u'15452', ([u'152', 5], [u'23589', 0.32732683535398865]))]

In [30]:
grouped_vote_rdd=(vote_rdd
                    .map(lambda x: ((x[1][0][0],x[1][1][0]),(x[1][0][1],x[1][1][1])))
                    .groupByKey())
grouped_vote_rdd.map(lambda x: (x[0],list(x[1]))).take(10)

[((u'152', u'27559'),
  [(8, 1.0),
   (8, 1.0),
   (10, 0.8660254037844448),
   (9, 1.0),
   (3, 1.0),
   (8, 1.0),
   (9, 0.899228803025897),
   (7, 1.0),
   (10, 1.0),
   (6, 1.0),
   (10, 0.8660254037844448)]),
 ((u'51', u'27091'), [(6, 1.0)]),
 ((u'122', u'23109'), [(8, 1.0)]),
 ((u'111', u'11157'), [(7, 1.0), (10, 1.0)]),
 ((u'105', u'33523'), [(10, 0.5399492471560386)]),
 ((u'281', u'25088'), [(9, 1.0)]),
 ((u'164', u'12851'), [(7, 1.0)]),
 ((u'187', u'31268'), [(5, 1.0)]),
 ((u'179', u'31144'), [(7, 1.0)]),
 ((u'152', u'36917'), [(7, 1.0)])]

In [32]:
def compute_rec (x):
    num=sum(i[0]*i[1] for i in x[1])
    den=sum(i[1] for i in x[1])
    if den==0:
        return 0
    return x[0][0],(x[0][1],float(num)/den)



rec_rdd=(grouped_vote_rdd
         .map(compute_rec)
         .groupByKey()
         .map(lambda x: (x[0],sorted(list(y for y in x[1]),key=lambda x: -x[1]))))
rec_rdd.take(10)

[(u'164',
  [(u'12313', 10.000000000000002),
   (u'30758', 10.000000000000002),
   (u'6773', 10.000000000000002),
   (u'30173', 10.000000000000002),
   (u'24786', 10.0),
   (u'35300', 10.0),
   (u'17205', 10.0),
   (u'36835', 10.0),
   (u'27271', 10.0),
   (u'27488', 10.0),
   (u'20283', 10.0),
   (u'24834', 10.0),
   (u'17717', 10.0),
   (u'8576', 10.0),
   (u'8394', 10.0),
   (u'25886', 10.0),
   (u'17597', 10.0),
   (u'25512', 10.0),
   (u'7733', 10.0),
   (u'11157', 10.0),
   (u'1304', 10.0),
   (u'21765', 10.0),
   (u'1438', 10.0),
   (u'27686', 10.0),
   (u'4228', 10.0),
   (u'2578', 10.0),
   (u'3456', 10.0),
   (u'14598', 10.0),
   (u'18178', 10.0),
   (u'30042', 10.0),
   (u'3674', 10.0),
   (u'24892', 10.0),
   (u'30440', 10.0),
   (u'20243', 10.0),
   (u'35410', 10.0),
   (u'2284', 10.0),
   (u'34351', 10.0),
   (u'36179', 10.0),
   (u'30616', 10.0),
   (u'36957', 10.0),
   (u'18016', 10.0),
   (u'1574', 10.0),
   (u'21678', 10.0),
   (u'12263', 10.0),
   (u'34614', 10.0),
 